# A09 Bagging

In [39]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

default = pd.read_csv("Default.csv")
default.head()

default["default_bin"] = (default["default"] == "Yes").astype(int)
default["student_bin"] = (default["student"] == "Yes").astype(int)

X_def = default[["student_bin", "balance", "income"]].values
y_def = default["default_bin"].values

print("default.shape:", default.shape)

default.shape: (10000, 6)


### Árbol de decisión simple

In [42]:
tree_clf = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,        
    random_state=123
)

tree_clf.fit(X_def, y_def)

y_pred_tree = tree_clf.predict(X_def)
acc_tree = accuracy_score(y_def, y_pred_tree)
print("Accuracy árbol de decisión:", acc_tree)

Accuracy árbol de decisión: 1.0


### Bagging manual con 5000 bootstraps

In [8]:
n_boot = 5000
n_def  = default.shape[0]

rng = np.random.default_rng(123)

accs_boot = np.zeros(n_boot)

for b in range(n_boot):
    idx = rng.integers(0, n_def, n_def)
    
    X_b = X_def[idx]
    y_b = y_def[idx]
    
    tree_b = DecisionTreeClassifier(
        criterion="gini",
        max_depth=None,
        random_state=b  
    )
    
    tree_b.fit(X_b, y_b)

    y_pred_b = tree_b.predict(X_def)
    accs_boot[b] = accuracy_score(y_def, y_pred_b)

print("Bagging con DecisionTree 5000")
print("Accuracy medio :", accs_boot.mean())
print("Desv. estándar :", accs_boot.std(ddof=1))

Bagging con DecisionTree 5000
Accuracy medio : 0.9836351999999999
Desv. estándar : 0.0010850845320473142


### RandomForest

In [9]:
rf_clf = RandomForestClassifier(
    n_estimators=5000,     
    bootstrap=True,        
    max_features="sqrt",   
    random_state=123,
    n_jobs=-1              
)

rf_clf.fit(X_def, y_def)

y_pred_rf = rf_clf.predict(X_def)
acc_rf = accuracy_score(y_def, y_pred_rf)
print("Accuracy RandomForest bagging con 5000 árboles:", acc_rf)

Accuracy RandomForest bagging con 5000 árboles: 1.0


### AUC de la regresión logística

In [27]:
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
import pandas as pd

default = pd.read_csv("Default.csv")

default["default_bin"] = (default["default"] == "Yes").astype(int)
default["student_bin"] = (default["student"] == "Yes").astype(int)

X_def = default[['student_bin','balance','income']]
X_def = sm.add_constant(X_def)      
y_def = default['default_bin']

logit_def = sm.Logit(y_def, X_def).fit()

print(logit_def.summary())

Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:            default_bin   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Mon, 24 Nov 2025   Pseudo R-squ.:                  0.4619
Time:                        17:19:50   Log-Likelihood:                -785.77
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                3.257e-292
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const         -10.8690      0.492    -22.079      0.000     -11.834      -9.904
student_bin    -0.6468   

In [29]:
y_pred_logit = logit_def.predict()

auc_logit = roc_auc_score(y_def, y_pred_logit)
print("AUC regresión logística:", auc_logit)

AUC regresión logística: 0.9495581233452343


### AUC de bagging

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

default = pd.read_csv("Default.csv")
default["default_bin"] = (default["default"] == "Yes").astype(int)
default["student_bin"] = (default["student"] == "Yes").astype(int)

X_def = default[["student_bin", "balance", "income"]].values   
y_def = default["default_bin"].values                          

print("default.shape:", default.shape)
print("X_def.shape:", X_def.shape)
print("y_def.shape:", y_def.shape)

default.shape: (10000, 6)
X_def.shape: (10000, 3)
y_def.shape: (10000,)


In [36]:
n_boot = 5000
n_def  = X_def.shape[0]      

rng = np.random.default_rng(123)
auc_bagging = np.zeros(n_boot)

for b in range(n_boot):
    idx = rng.integers(0, n_def, n_def)   

    X_b = X_def[idx, :]   
    y_b = y_def[idx]

    tree_b = DecisionTreeClassifier(
        criterion="gini",
        max_depth=None,
        random_state=b
    )
    tree_b.fit(X_b, y_b)

    y_pred_prob_b = tree_b.predict_proba(X_def)[:, 1]
    auc_bagging[b] = roc_auc_score(y_def, y_pred_prob_b)

print("AUC Bagging promedio:", auc_bagging.mean())
print("Desv. estándar AUC Bagging:", auc_bagging.std(ddof=1))

AUC Bagging promedio: 0.8749746565433748
Desv. estándar AUC Bagging: 0.012095144482348583


### Comparación 

In [45]:
moda_auc = pd.Series(auc_bagging).mode()

print("Moda de los AUC del bagging:")
print(moda_auc.values)

Moda de los AUC del bagging:
[0.87273909 0.87563865 0.8770367 ]


In [57]:
moda_bagging = pd.Series(auc_bagging).mode().values
moda_logit = pd.Series([auc_logit]).mode().values

tabla_moda = pd.DataFrame({
    "Modelo": ["Regresión Logística", "Bagging 5000"],
    "Moda AUC": [moda_logit[0], moda_bagging[0]]
})

tabla_moda


,Modelo,Moda AUC
0,Regresión Logística,0.949558
1,Bagging 5000,0.872739


### Conclusiones

Los resultados muestran que la regresión logística es el modelo que mejor funciona para predecir el default. Su AUC es alrededor de 0.95, lo que significa que separa muy bien a los clientes que sí harán default de los que no. En cambio, el bagging con 5000 árboles tiene un AUC promedio de 0.87, así que aunque mejora la estabilidad y reduce la variación entre muestras, sigue siendo menos preciso que la regresión logística.

Por otro lado, el Random Forest logró una exactitud de 1.0, pero esto sucede porque está aprendiendo “demasiado” del mismo conjunto de datos; es decir, se está sobreajustando. Por eso su resultado no es confiable cuando no se prueba con datos nuevos. En resumen, para este caso la regresión logística es el modelo más equilibrado, más estable y el que mejor predice, mientras que los árboles tienden a sobreajustarse si no se validan correctamente.